In [1]:
import pathlib
import tensorflow as tf
import PIL.Image

data_dir = r'C:\Users\horin\Desktop\возращение блюдного сына\Нейросети\archive\flowers'

batch_size = 32
img_height = 150
img_weight = 150

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 3,
    image_size = (img_height, img_weight),
    batch_size = batch_size
)


Found 4317 files belonging to 5 classes.
Using 3454 files for training.


In [2]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 3,
    image_size = (img_height, img_weight),
    batch_size = batch_size
)

Found 4317 files belonging to 5 classes.
Using 863 files for validation.


In [3]:
class_names = train_ds.class_names
print (class_names)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [4]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 150, 150, 3)
(32,)


In [5]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
num_classes = 5
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

num_classes = 5

model = Sequential([
    layers.Rescaling(1./255, input_shape = (img_height, img_weight, 3)),
    layers.Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same',activation ='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters =64, kernel_size = (3,3),padding = 'Same',activation ='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Flatten(),
    layers.Dense(512),
    layers.Activation('relu'),
    layers.Dense(5, activation = "softmax")
])

In [7]:
model.compile(
    optimizer = 'adam', 
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics = ['accuracy'] )

In [8]:
model.fit(
train_ds,
validation_data=val_ds,
epochs = 10)

Epoch 1/10


C:\Users\horin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


108/108 [==============================] - 143s 1s/step - loss: 1.2705 - accuracy: 0.4624 - val_loss: 1.0802 - val_accuracy: 0.5852
Epoch 2/10
108/108 [==============================] - 86s 792ms/step - loss: 1.0153 - accuracy: 0.6005 - val_loss: 1.0130 - val_accuracy: 0.6199
Epoch 3/10
108/108 [==============================] - 86s 797ms/step - loss: 0.8912 - accuracy: 0.6569 - val_loss: 1.0159 - val_accuracy: 0.6118
Epoch 4/10
108/108 [==============================] - 88s 811ms/step - loss: 0.7536 - accuracy: 0.7122 - val_loss: 1.0121 - val_accuracy: 0.6304
Epoch 5/10
108/108 [==============================] - 86s 798ms/step - loss: 0.6220 - accuracy: 0.7672 - val_loss: 1.0499 - val_accuracy: 0.6350
Epoch 6/10
108/108 [==============================] - 87s 805ms/step - loss: 0.5089 - accuracy: 0.8135 - val_loss: 1.2411 - val_accuracy: 0.6327
Epoch 7/10
108/108 [==============================] - 88s 813ms/step - loss: 0.4581 - accuracy: 0.8312 - val_loss: 1.3729 - val_accuracy: 0.621

In [11]:
from PIL import Image

image = Image.open(r'C:\Users\horin\Desktop\A_rose_flower.jpg')
image.load()
image.show()

In [12]:
import numpy as np

image = image.resize((150, 150))
image_tensor = tf.convert_to_tensor(np.array(image), dtype = tf.float32)
image_tensor = tf.expand_dims(image_tensor, 0)
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

predictions = probability_model.predict(image_tensor, use_multiprocessing=True)
predictions[0]
print("{} with a {:.2f} percent"
      .format(class_names[np.argmax(predictions[0])], 100* np.max(predictions[0]))
     )

1/1 [==============================] - 0s 388ms/step
tulip with a 40.46 percent
